In [1]:
import sys 
sys.path.append("../../../")

In [2]:
import matplotlib.pyplot as plt 
import pandas
from src.drift import detection
import pickle

In [3]:
%load_ext line_profiler 
%load_ext memory_profiler

Defining constant values

In [4]:
NEW_DATA_URL = str(input("Load compressed version of fresh dataset"))
OLD_DATA_URL = str(input("Load compressed version of old dataset"))

Load compressed version of fresh dataset ../data/compressed_data/category_dataset.pkl
Load compressed version of old dataset ../data/compressed_data/category_dataset.pkl


Loading dataset

In [5]:
try:
    new_news_applications = pickle.load(open(NEW_DATA_URL, mode='rb')) 
    old_news_applications = pickle.load(open(OLD_DATA_URL, mode='rb'))
except(FileNotFoundError):
    raise SystemExit("Failed to load datasets, check whether they exists")

In [6]:
new_news_applications

,link,headline,category,short_description,authors,date
0,https://www.huffpost.com/entry/covid-boosters-...,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP",2022-09-23
1,https://www.huffpost.com/entry/american-airlin...,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss,2022-09-23
2,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,2022-09-23
3,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,2022-09-23
4,https://www.huffpost.com/entry/amy-cooper-lose...,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Nina Golgowski,2022-09-22
...,...,...,...,...,...,...
209522,https://www.huffingtonpost.com/entry/rim-ceo-t...,RIM CEO Thorsten Heins' 'Significant' Plans Fo...,TECH,Verizon Wireless and AT&T are already promotin...,"Reuters, Reuters",2012-01-28
209523,https://www.huffingtonpost.com/entry/maria-sha...,Maria Sharapova Stunned By Victoria Azarenka I...,SPORTS,"Afterward, Azarenka, more effusive with the pr...",,2012-01-28
209524,https://www.huffingtonpost.com/entry/super-bow...,"Giants Over Patriots, Jets Over Colts Among M...",SPORTS,"Leading up to Super Bowl XLVI, the most talked...",,2012-01-28
209525,https://www.huffingtonpost.com/entry/aldon-smi...,Aldon Smith Arrested: 49ers Linebacker Busted ...,SPORTS,CORRECTION: An earlier version of this story i...,,2012-01-28


In [7]:
new_news_applications.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209527 entries, 0 to 209526
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   link               209527 non-null  object        
 1   headline           209527 non-null  object        
 2   category           209527 non-null  category      
 3   short_description  209527 non-null  object        
 4   authors            209527 non-null  object        
 5   date               209527 non-null  datetime64[ns]
dtypes: category(1), datetime64[ns](1), object(4)
memory usage: 8.2+ MB


In [8]:
old_news_applications

,link,headline,category,short_description,authors,date
0,https://www.huffpost.com/entry/covid-boosters-...,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP",2022-09-23
1,https://www.huffpost.com/entry/american-airlin...,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss,2022-09-23
2,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,2022-09-23
3,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,2022-09-23
4,https://www.huffpost.com/entry/amy-cooper-lose...,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Nina Golgowski,2022-09-22
...,...,...,...,...,...,...
209522,https://www.huffingtonpost.com/entry/rim-ceo-t...,RIM CEO Thorsten Heins' 'Significant' Plans Fo...,TECH,Verizon Wireless and AT&T are already promotin...,"Reuters, Reuters",2012-01-28
209523,https://www.huffingtonpost.com/entry/maria-sha...,Maria Sharapova Stunned By Victoria Azarenka I...,SPORTS,"Afterward, Azarenka, more effusive with the pr...",,2012-01-28
209524,https://www.huffingtonpost.com/entry/super-bow...,"Giants Over Patriots, Jets Over Colts Among M...",SPORTS,"Leading up to Super Bowl XLVI, the most talked...",,2012-01-28
209525,https://www.huffingtonpost.com/entry/aldon-smi...,Aldon Smith Arrested: 49ers Linebacker Busted ...,SPORTS,CORRECTION: An earlier version of this story i...,,2012-01-28


Calculating statistics

In [10]:
stat_summary = detection.compare_datasets(
    old_data=old_news_applications,
    new_data=new_news_applications
)

AttributeError: module 'scipy.stats' has no attribute 'FitDataError'

Checking results

In [9]:
stat_summary

defaultdict(dict,
            {'link': {'null_exceed': False},
             'headline': {'null_exceed': False},
             'category': {'null_exceed': False},
             'short_description': {'null_exceed': False},
             'authors': {'null_exceed': False},
             'date': {'null_exceed': False}})

Comparing numerical distributions

In [10]:
def compare_distributions(old_dataset: pandas.DataFrame, new_dataset: pandas.DataFrame):
    """
    Function plots numerical distributions of old and new data
    using matplotlib library

    Args:
        old_dataset (pandas.DataFrame) old version of the original dataset
        new_dataset (pandas.DataFrame) new version of the original dataset
    """
    if len(new_dataset.columns) == 0: return 
    _, axs = plt.subplots(len(new_dataset.columns), 2)

    if len(new_dataset.columns) == 1:
        axs[0].hist(x=old_dataset[feature])
        axs[0].set_title("Old " + feature + " Distribution")
        
        axs[1].hist(x=new_dataset[feature])
        axs[1].set_title("New " + feature + " Distribution")
    else:
        for idx, feature in enumerate(new_dataset.columns):
            
            axs[idx, 0].hist(x=old_dataset[feature])
            axs[idx, 0].set_title("Old " + feature + " Distribution")
            
            axs[idx, 1].hist(x=new_dataset[feature])
            axs[idx, 1].set_title("New " + feature + " Distribution")
            
    plt.title("Numerical Data Distributions")
    plt.subplots_adjust(wspace=0.5, hspace=0.5)
    plt.show()

numerical_features = new_news_applications.select_dtypes(include='number').columns

# Visualizing distributions

compare_distributions(
    old_dataset=old_news_applications[numerical_features],
    new_dataset=new_news_applications[numerical_features]
)